In [170]:
# !pip install catboost

In [171]:
import pandas as pd
import sys
# from google.colab import files
# import io 
import re
import datetime
import ast

from sklearn.model_selection import train_test_split

# df = pd.read_csv("Kickstarter.csv")

"""
The dataset can be found here:
https://webrobots.io/kickstarter-datasets/
https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2021-06-17T03_20_03_179Z.zip
"""

'\nThe dataset can be found here:\nhttps://webrobots.io/kickstarter-datasets/\nhttps://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2021-06-17T03_20_03_179Z.zip\n'

In [172]:
# df = df.drop(columns=["currency_symbol",
#                  "country_displayable_name",
#                  "created_at",
#                  "creator",
#                  "country",
#                  "urls",
#                  "currency_trailing_code",
#                  "friends",
#                  "fx_rate",
#                  "id",
#                  "is_starred",
#                  "name",
#                  "permissions"], axis = 1)

In [173]:
# df.head(3)

Do we want to combine find the length of the campaign from (created at and deadline) to see if there is value there. 

keep location, clean and ckeep city name



In [174]:
# def clean_data(text):
#     email_regex = r"\S*@\S*\s?"
#     non_alpha = "[^a-zA-Z0-9 ]"
#     multi_white_spaces = "[ ]{2,}"
#     text = re.sub(email_regex, "", text)
#     text = re.sub(non_alpha, "", text)
#     text = re.sub(multi_white_spaces, "", text)
#     return text.lower().lstrip().rstrip()

# Load The Dataset

In [175]:
# file_path = 'Downloads/Kickstarter_2021-06-17T03_20_03_179Z/Kickstarter001.csv'
file_path = "Kickstarter.csv"
# file_path_small_test = 'small_file.csv'


def clean_data(file_path):
    df = pd.read_csv(file_path)
    
    def change_to_time_series(item):
        item = datetime.datetime.fromtimestamp(item)
        return item
    
    # change the 'created-at', 'deadline', 'state_changed_at' columns
    # to time series columns
    time_series_columns = ['created_at', 'deadline', 'state_changed_at', 'launched_at']
    for column in time_series_columns:
        df[column] = df[column].apply(change_to_time_series)
    
    # dropping columns with all null values
    # for reference these the column names:
    # ['friends', 'is_backing', 'is_starred', 'permissions']
    all_null_columns = df.isnull().sum() == len(df)
#     to_drop = all_null_columns[all_null_columns == True].index
    to_drop = ['friends', 'is_backing', 'is_starred', 'permissions']
    df = df.drop(columns=to_drop, axis=1)
    
    df = df.drop(columns=["currency_symbol",
                          "country_displayable_name",
                          "creator",
                          "country",
                          "urls",
                          "source_url",
                          "currency_trailing_code",
                          "fx_rate",
                          "id",
                          "name",
                          "disable_communication",
                          "photo", # has url for photos might be useful later 
                          "usd_type", 
                          'converted_pledged_amount',
                          'pledged',
                          'usd_exchange_rate',
                          'static_usd_rate',
                          'currency', # dropping due to have pledged in USD
                          'current_currency'  # dropping due to have pledged in USD
                         ], axis=1)
    
    def extract_dictionary_info(item):
        """ Use the apply method with the column name.
        Takes in dictionary in string form,
        converts it into a dictionary, and
        returns info_type requested in a new column.
        """
        my_dic = ast.literal_eval(item)
        return my_dic

    cat_df = df['category'].apply(extract_dictionary_info).apply(pd.Series)
    cat_df['parent_name'].loc[(cat_df.parent_name.isnull() == True)& (cat_df['name'] == 'Dance')] = 'Dance'
    cat_df['parent_name'].loc[(cat_df.parent_name.isnull() == True)& (cat_df['name'] == 'Photography')] = 'Photography'
    cat_df['parent_name'].loc[(cat_df.parent_name.isnull() == True)& (cat_df['name'] == 'Games')] = 'Games'
    
    df['category_2'] = cat_df.parent_name    
    df = pd.concat([df,pd.get_dummies(df['category_2'], dtype = float)], axis=1)
    df.drop(columns=['category', 'category_2'], inplace=True)
    
    # columns I can't get to work
    # in dictionary format need to work on OR text data
    # text_data need to vectorize: 'blurb
    drop_for_now = ['blurb', 'location', 'profile', 'slug']
    df.drop(columns=drop_for_now, inplace=True)
    
    
    # creating our target variable, we had the four sub-categories:
    # successful, failed, canceled, live
    # turn it into a binary variable
    df['target'] = 0
    df['target'].loc[df.state == 'successful'] = 1
    df.drop(columns='state', inplace=True)
    
    # changing boolean to integers
    df['spotlight_2'] = 0
    df['spotlight_2'].loc[df.spotlight == True] = 1

    df['staff_pick_2'] = 0
    df['staff_pick_2'].loc[df.staff_pick == True] = 1

    df['is_starrable_2'] = 0
    df['is_starrable_2'].loc[df.is_starrable == True] = 1
    df.drop(columns=['spotlight', 'staff_pick', 'is_starrable', "spotlight_2", "backers_count"],
           inplace=True)


    
    # removing the launched_at and state_changed_at time series columns
    # add at your discretion
    time_series = ['created_at', 'deadline','state_changed_at', 'launched_at', "usd_pledged"] # Drop USD pledged for leakage, duh 
    df.drop(columns=time_series, inplace=True)
    
    
    return df

In [176]:
# Reading in the data and passing through the cleaning function
df = clean_data(file_path)
# df = clean_data(file_path_small_test)
df.head(3)


C:\Users\iankn\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\iankn\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\iankn\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

,goal,Dance,Fashion,Film & Video,Games,target,staff_pick_2,is_starrable_2
0,15000.0,0.0,0.0,0.0,1.0,0,0,1
1,10000.0,0.0,0.0,1.0,0.0,1,0,0
2,3000.0,1.0,0.0,0.0,0.0,1,0,0


In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3662 entries, 0 to 3661
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   goal            3662 non-null   float64
 1   Dance           3662 non-null   float64
 2   Fashion         3662 non-null   float64
 3   Film & Video    3662 non-null   float64
 4   Games           3662 non-null   float64
 5   target          3662 non-null   int64  
 6   staff_pick_2    3662 non-null   int64  
 7   is_starrable_2  3662 non-null   int64  
dtypes: float64(5), int64(3)
memory usage: 229.0 KB


# Split the Data

In [178]:
df.corr()

,goal,Dance,Fashion,Film & Video,Games,target,staff_pick_2,is_starrable_2
goal,1.000000,-0.027358,-0.016564,0.050612,-0.011118,-0.056292,-0.010174,-0.005652
Dance,-0.027358,1.000000,-0.281510,-0.436077,-0.368743,0.239547,0.208503,-0.148809
Fashion,-0.016564,-0.281510,1.000000,-0.282603,-0.238966,-0.337926,-0.157599,-0.088135
Film & Video,0.050612,-0.436077,-0.282603,1.000000,-0.370174,0.070525,0.026574,-0.144157
Games,-0.011118,-0.368743,-0.238966,-0.370174,1.000000,-0.048008,-0.119936,0.390873
target,-0.056292,0.239547,-0.337926,0.070525,-0.048008,1.000000,0.215234,-0.356878
staff_pick_2,-0.010174,0.208503,-0.157599,0.026574,-0.119936,0.215234,1.000000,-0.058831
is_starrable_2,-0.005652,-0.148809,-0.088135,-0.144157,0.390873,-0.356878,-0.058831,1.000000


In [179]:
# Split the data to avoid leakage
x = df.drop(columns=['target'])
y = df['target']

In [180]:
# Create a training and validation set
# Smaller test size is acceptable due to the amount of data available
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=.1, random_state=42)

print('x_train:', len(X_train))
print('x_val:', len(X_val))
print('y_train:',y_train.shape)
print('y_val:', y_val.shape)

x_train: 3295
x_val: 367
y_train: (3295,)
y_val: (367,)


# Set Model Baseline

In [195]:
baseline = y_train.value_counts(normalize = True).max()
print("The baseline score for our dataset is:", baseline)

The baseline score for our dataset is: 0.6937784522003035


# Explore Models

In [181]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.callbacks import TensorBoard

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# required for compatibility bewteen sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

## Configure a Neural Network ##

In [182]:
# Define input dimensions to avoid hidden layer issues
input_dims = X_train.shape[1]
input_dims

7

## Create Model Function ##
This function should help you save time on creating more models.

In [183]:
# Function to create model, required for KerasClassifier
def create_model(units = 128, optimizer = "adam", activation = "sigmoid"):
    """"
    Returns a complied keras model 
    
    Parameters
    ----------
    units: int 
        number of neruons/nodes/units to use in each hidden layer
        
    Returns
    -------
    model: keras object 
    """

    model = Sequential()
    model.add(Dense(units = 64, input_dim = input_dims, activation = activation))
    model.add(Dense(units = 32, activation = activation))
    model.add(Dense(units = 10, activation = activation))   
    model.add(Dense(1, activation="sigmoid")) # USE SIGMOID FOR BINARY CLASSIFICATION
    model.compile(loss="binary_crossentropy",
                    optimizer = "adam",
                    metrics=["accuracy"])
    return model

In [184]:
# Instantiate a base model for grid search
base_model = KerasClassifier(build_fn = create_model)

### Perform a Grid Search To Optimize Our Model 

In [185]:
# define the grid search parameters
param_grid = {'batch_size': [32],
              'epochs': [3],
              'units':[32, 64]}

In [186]:
# Create Grid Search
grid = GridSearchCV(estimator = base_model,
                    param_grid = param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Epoch 1/3
103/103 [==============================] - 0s 823us/step - loss: 0.6272 - accuracy: 0.6938
Epoch 2/3
103/103 [==============================] - 0s 804us/step - loss: 0.6168 - accuracy: 0.6938
Epoch 3/3
103/103 [==============================] - 0s 814us/step - loss: 0.6166 - accuracy: 0.6938
Best: 0.693780243396759 using {'batch_size': 32, 'epochs': 3, 'units': 32}
Means: 0.693780243396759, Stdev: 0.004719796940551231 with: {'batch_size': 32, 'epochs': 3, 'units': 32}
Means: 0.693780243396759, Stdev: 0.004719796940551231 with: {'batch_size': 32, 'epochs': 3, 'units': 64}


In [187]:
# Assign the best model 
best_nn_model = grid_result.best_estimator_
# Confirm the models params
best_nn_model.get_params()

{'batch_size': 32,
 'epochs': 3,
 'units': 32,
 'build_fn': <function __main__.create_model(units=128, optimizer='adam', activation='sigmoid')>}

In [188]:
    best_nn_model.fit(X_train, 
                        y_train, 
                        validation_data = (X_val,y_val),
                        verbose=2)

Epoch 1/3
103/103 - 1s - loss: 0.8463 - accuracy: 0.3193 - val_loss: 0.6906 - val_accuracy: 0.6676
Epoch 2/3
103/103 - 0s - loss: 0.6622 - accuracy: 0.6938 - val_loss: 0.6486 - val_accuracy: 0.6812
Epoch 3/3
103/103 - 0s - loss: 0.6361 - accuracy: 0.6938 - val_loss: 0.6356 - val_accuracy: 0.6812


### Check the models scores

In [189]:
best_nn_model_val_score = best_nn_model.score(X_val,y_val)
print("The best NN model validation score is:", best_nn_model_val_score)

12/12 [==============================] - 0s 818us/step - loss: 0.6356 - accuracy: 0.6812
The best NN model validation score is: 0.6811988949775696


# Create A Classifier Model 
Compare this model with our NN

In [190]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [191]:
clf_model = make_pipeline(
                        RandomForestClassifier(
                                                random_state= 42,
                                                ))

clf_model.fit(X_train, y_train)

Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

### Check the CLF models score

In [192]:
clf_model_val_score = clf_model.score(X_val,y_val)
print("The best CLF model validation score is:", clf_model_val_score)

The best CLF model validation score is: 0.773841961852861


In [193]:
# # Save the entire model as a SavedModel.
# !mkdir -p saved_model
# base_model.save('saved_model/base_model') 